<a href="https://colab.research.google.com/github/mansh2060/persist_venture_internship/blob/main/main_web.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from diffusers import StableDiffusionPipeline
import torch
import os
import sqlite3
from datetime import datetime
from PIL import Image

# Database setup
def setup_database():
    conn = sqlite3.connect("content_generation.db")
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS generated_content (
            user_id TEXT,
            prompt TEXT,
            image_paths TEXT,
            status TEXT,
            generated_at TEXT
        )
    """)
    conn.commit()
    return conn

# Generate image function
def generate_images_for_user(user_id, prompts, model_id="dreamlike-art/dreamlike-diffusion-1.0"):
    # Load the model
    pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float32, use_safetensors=True)
    pipe = pipe.to("cpu")

    # Directory to save generated content
    user_dir = os.path.join("generated_content", user_id)
    os.makedirs(user_dir, exist_ok=True)

    image_paths = []

    # Update status in the database to "Processing"
    for prompt in prompts:
        cursor.execute(
            "INSERT INTO generated_content (user_id, prompt, image_paths, status, generated_at) VALUES (?, ?, ?, ?, ?)",
            (user_id, prompt, None, "Processing", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
        )
        conn.commit()

    # Generate images
    for prompt in prompts:
        image = pipe(prompt).images[0]
        image_path = os.path.join(user_dir, f"{prompt.replace(' ', '_')}.png")
        image.save(image_path)
        image_paths.append(image_path)

        # Update the database with image path and status
        cursor.execute(
            "UPDATE generated_content SET image_paths = ?, status = ? WHERE user_id = ? AND prompt = ?",
            (image_path, "Completed", user_id, prompt)
        )
        conn.commit()

    # Notify user via console (or replace with email notification)
    print(f"User {user_id}: Your content has been generated and saved in {user_dir}.")

# Main function
def main():
    # Initialize database
    global conn, cursor
    conn = setup_database()
    cursor = conn.cursor()

    # Simulate taking user inputs
    user_id = input("Enter your user ID: ")
    prompts = []
    for i in range(5):
        prompt = input(f"Enter prompt {i + 1}: ")
        prompts.append(prompt)

    # Generate images
    generate_images_for_user(user_id, prompts)

    # Close the database connection
    conn.close()

if __name__ == "__main__":
    main()

Enter your user ID: manish
Enter prompt 1: A snowy mountain peak under a starry night sky, with a glowing aurora borealis
Enter prompt 2: A majestic waterfall cascading into a crystal-clear lake in a hidden valley
Enter prompt 3: A desert landscape at sunset, with sand dunes glowing orange and a caravan of camels in the distance
Enter prompt 4: A serene forest with sunlight streaming through the trees, surrounded by blooming flowers.
Enter prompt 5: An underwater coral reef teeming with colorful fish and vibrant marine life


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

User manish: Your content has been generated and saved in generated_content/manish.


In [ ]:
from flask import Flask, render_template, request, redirect, url_for, session
import sqlite3
import os
from datetime import datetime

app = Flask(__name__)
app.secret_key = 'manish123'

# Database setup
def setup_database():
    conn = sqlite3.connect('content_generation.db')
cursor = conn.cursor()
try:
    cursor.execute("BEGIN TRANSACTION;")
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS generated_content (
            user_id TEXT,
            prompt TEXT,
            image_paths TEXT,
            status TEXT,
            generated_at TEXT
        )
    """)
    cursor.execute("COMMIT;")
except sqlite3.DatabaseError as e:
    print(f"Error occurred: {e}")
    if conn:  # Check if connection is open
        conn.rollback()
finally:
    if conn:
        conn.close()

setup_database()

# Helper functions
def log_user_action(user_id, action):
    conn = sqlite3.connect("content_generation.db")
    cursor = conn.cursor()
    cursor.execute(
        "INSERT INTO user_logs (user_id, action, timestamp) VALUES (?, ?, ?)",
        (user_id, action, datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    )
    conn.commit()
    conn.close()

def get_user_content(user_id):
    conn = sqlite3.connect("content_generation.db")
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM generated_content WHERE user_id = ?", (user_id,))
    content = cursor.fetchall()
    conn.close()
    return content

# Routes
@app.route('/')
def home():
    return render_template('login.html')

@app.route('/login', methods=['POST'])
def login():
    user_id = request.form['user_id']
    session['user_id'] = user_id
    log_user_action(user_id, "Login attempt")
    return redirect(url_for('dashboard'))

@app.route('/dashboard')
def dashboard():
    if 'user_id' not in session:
        return redirect(url_for('home'))

    user_id = session['user_id']
    content = get_user_content(user_id)

    if not content or all(item[3] == "Processing" for item in content):
        log_user_action(user_id, "Viewed dashboard - Content processing")
        return render_template('processing.html', user_id=user_id)


    images = [item[2] for item in content if item[2] and item[3] == "Completed"]
    log_user_action(user_id, "Viewed dashboard - Content ready")
    return render_template('dashboard.html', user_id=user_id, images=images)

@app.route('/logout')
def logout():
    user_id = session.pop('user_id', None)
    if user_id:
        log_user_action(user_id, "Logout")
    return redirect(url_for('home'))


if __name__ == "__main__":
    app.run(debug=True)

